Ce script vise à utiliser l'API Légifrance, disponible via PISTE. 

# 1/ Mise en place
Il nous faut dans un premier temps obtenir notre token : celui-ci s'obtient grâce aux identifiants générés sur le portail PISTE lorsqu'on a accepté les CGU de l'API Légifrance.

In [22]:
import requests
import pandas as pd

# Prompt the user to input client_id and client_secret
# (J'imagine que hardcoder ses identifiants est une mauvaise pratique)
client_id = input("Enter your client_id: ")  # client_id generated by PISTE
client_secret = input("Enter your client_secret: ")  # client_secret generated by PISTE

url = "https://sandbox-oauth.piste.gouv.fr/api/oauth/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

# Store the access token in a variable
access_token = None
if response.status_code == 200:
    access_token = response.json().get("access_token")
else:
    print("Error:", response.status_code, response.text)


# 2/ Accès à la documentation Swagger

In [2]:
import json
import requests

# Charger la documentation Swagger depuis une URL
url = "https://github.com/user-attachments/files/17714249/Legifrance.json"
response = requests.get(url)
swagger_doc = response.json()

# Afficher des informations générales
print("Titre:", swagger_doc['info']['title'])
print("Version:", swagger_doc['info']['version'])
print("Description:", swagger_doc['info'].get('description', 'Pas de description disponible'))

# Parcourir les chemins (endpoints)
print("\nEndpoints disponibles :")
for path, methods in swagger_doc['paths'].items():
    print(f"\nPath : {path}")
    for method, details in methods.items():
        print(f"  Méthode : {method.upper()}")
        print("    Description :", details.get('description', 'Pas de description disponible'))

        # Afficher les paramètres de chaque méthode
        if 'parameters' in details:
            print("    Paramètres :")
            for param in details['parameters']:
                param_type = param.get('type', 'inconnu')  # Définit le type à "inconnu" si la clé "type" n'existe pas
                print(f"      - {param['name']} (type: {param_type}) - {'Obligatoire' if param.get('required') else 'Optionnel'}")

        # Afficher les réponses possibles
        print("    Réponses :")
        for status_code, response in details['responses'].items():
            print(f"      - Code {status_code}: {response.get('description', 'Pas de description')}")


Titre: Légifrance
Version: 2.4.2
Description: 
Afin de faciliter la réutilisation des données juridiques, la DILA met à disposition du public une API de Légifrance.
Ce document a pour objectif de décrire les méthodes de l'API Légifrance, présentées à l'aide du framework Swagger.
Vous pouvez également télécharger une [documentation complémentaire](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/description-des-tris-et-filtres-de-l-api.xlsx) ainsi que des [exemples d’utilisation](https://www.legifrance.gouv.fr/contenu/Media/Files/pied-de-page/exemples-d-utilisation-de-l-api.docx) de l'API.

Cet accès vous permet d’expérimenter l'API sur un environnement de test et de vous familiariser avec les différentes méthodes.

À noter :
- L'environnement de test peut être indisponible.
- Les appels sont limités par quotas.

Les mises à jour correctives et évolutives sont réalisées dans un premier temps sur cet environnement.
L'API mise à disposition correspond à celle utilisée par l

# 3/ Requêtes pour les prénoms au JORF
Notre objectif, exploratoire dans un premier temps, est d'avoir pour l'année 2024 l'ensemble des nominations. Il nous faudra sans doute récupérer le NOR, qui nous renseignera le ministère et l'année. 

Pour l'instant, on voit en regardant l'output qu'il n'y a pas de tag préexistant nous facilitant beaucoup la tâche : il serait chouette que certains codes soient stables, mais j'en doute.

In [3]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Afin de comprendre le format d'un output de nomination, utilisons le NOR d'une nomination donnée
payload = {
    "nor": "ECOP2427670A"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    print(data)  # Affiche le contenu brut de la réponse pour analyse
else:
    print(f"Erreur {response.status_code}: {response.text}")

{'executionTime': 2, 'dereferenced': False, 'id': 'JORFTEXT000050397779_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000050397779', 'title': "Arrêté du 18 octobre 2024 portant nominations au cabinet du ministre de l'économie, des finances et de l'industrie", 'nor': 'ECOP2427670A', 'eli': None, 'alias': None, 'jorfText': 'JORF n°0255 du 26 octobre 2024', 'jurisState': 'Sans état juridique', 'visa': "<p align='left'><br/>Le ministre de l'économie, des finances et de l'industrie,<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000034938597&categorieLien=cid' title='Décret n°2017-1098 du 14 juin 2017'>décret n° 2017-1098 du 14 juin 2017</a> relatif aux collaborateurs du Président de la République et des membres du Gouvernement ;<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000050251629&categorieLien=cid' title='Décret n°2024-892 du 23 septembre 2024'>décret n° 2024-892 du 23 septembre 2024</a> relatif à la composition des cabinets ministériels ;<br/>Vu le <a href='/affichTe

Puis, avec le module search

In [4]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/search/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}


payload = {
    "recherche": {
        "filtres": [
            {
                "valeurs": ["ARRETE"],  # Filtrer uniquement par arrêté, ce qui inclut les nominations
                "facette": "NATURE"
            },
            {
                "dates": {
                    "start": "2024-01-01",
                    "end": "2024-12-31"
                },
                "facette": "DATE_SIGNATURE"
            }
        ],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": False,
        "secondSort": "ID",
        "champs": [
            {
                "criteres": [
                    {
                        "valeur": "cabinet ministre",  # Critère pour filtrer les nominations dans les cabinets ministériels
                        "operateur": "ET",
                        "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"
                    }
                ],
                "operateur": "ET",
                "typeChamp": "TITLE"
            }
        ],
        "pageSize": 10,  # Ajustez selon vos besoins
        "operateur": "ET",
        "typePagination": "DEFAUT",
        "pageNumber": 1
    },
    "fond": "LODA_DATE"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    # Extraction des NOR des arrêtés
    nors = [item['nor'] for item in data.get('results', [])]
    print("NOR des nominations en 2024:", nors)
else:
    print("Erreur lors de la requête:", response.status_code, response.text)



NOR des nominations en 2024: ['PRMX2430415A', 'MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'MOMC2429076A', 'PRMX2429339A', 'PRMX2429154A', 'PRMX2428951A', 'PRMX2428841A', 'MOMC2427547A']


Désormais, pour l'ensemble de ces extraits du JO, identifiés par les NOR, nous allons essayer de récuperer les informations souhaitées.

In [ ]:
import requests
import pandas as pd
import re

## CECI FONCTIONNE POUR UN NOR DONNE : a poursuivre et améliorer le loop en s'inspirant de ce code. 
# API details
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Payload for the first NOR
payload = {
    "nor": "ECOP2427670A"
}

# Send the API request
response = requests.post(url, headers=headers, json=payload)

# Process the response
if response.status_code == 200:
    data = response.json()

    # Extract ministry name
    ministry_name_match = re.search(r"cabinet du (.*?)(,|$)", data.get("title", ""), re.IGNORECASE)
    ministry_name = ministry_name_match.group(1).strip() if ministry_name_match else "Cabinet"

    # Extract minister's name
    minister_name_match = re.search(r"Fait le .*?\.<br/>(.*?)</p>", data.get("signers", ""), re.DOTALL)
    minister_name = minister_name_match.group(1).strip() if minister_name_match else None

    # Extract nominees information
    articles = data.get("articles", [])
    nominees = []
    for article in articles:
        content = article.get("content", "")
        matches = re.findall(
            r"(M\.|Mme) ([\w\-]+) ([\w\-]+)(?:,| est nommée?| est nommé) (.*?)(?:,|\.| à compter du) à compter du ([\w\s\-éà]+)(?:\.|;|<br/>)",
            content,
            re.IGNORECASE
        )
        for match in matches:
            genre, prenom, nom, titre, date = match
            nominees.append({
                "Genre": genre,
                "Prénom": prenom,
                "Nom": nom,
                "Titre": titre.strip(),
                "Date de début": date.strip(),
                "Ministère": ministry_name,
                "Ministre": minister_name,
                "NOR": payload["nor"]
            })

    # Convert to DataFrame
    df = pd.DataFrame(nominees)
    print(df)  # Display the DataFrame

else:
    print(f"Error {response.status_code}: {response.text}")


  Genre          Prénom          Nom  \
0    M.         Sylvain  Maisonneuve   
1   Mme      Christelle       Delmas   
2   Mme      Marie-Anne     Lavergne   
3    M.        Augustin    Aoudjhane   
4    M.         Edouard    Montchamp   
5    M.          Julien    Franchina   
6    M.  Pierre-Léonard      Rouzaud   

                                               Titre      Date de début  \
0  conseiller en charge des entreprises, de la co...     4 octobre 2024   
1                               conseillère spéciale     3 octobre 2024   
2  conseillère financement de l'économie et outre...     7 octobre 2024   
3                       directeur de cabinet adjoint          7 octobre   
4                         chef du pôle communication         14 octobre   
5                                  conseiller presse  23 septembre 2024   
6                conseiller Etat actionnaire, espace    14 octobre 2024   

                Ministère Ministre           NOR  
0  ministre de l'économie  

In [ ]:
import re

# List of NORs to process defined above

# API details
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Initialize lists to track successes and errors
all_nominees = []
error_nors = []
# Loop through each NOR in the list
for nor in nors:
    payload = {"nor": nor}
    try:
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            data = response.json()

            # Extract ministry name
            ministry_name_match = re.search(r"cabinet du (.*?)(,|$)", data.get("title", ""), re.IGNORECASE)
            ministry_name = ministry_name_match.group(1).strip() if ministry_name_match else "Cabinet"

            # Extract minister's name
            minister_name_match = re.search(r"Fait le .*?\.<br/>(.*?)</p>", data.get("signers", ""), re.DOTALL)
            minister_name = minister_name_match.group(1).strip() if minister_name_match else None

            # Extract nominees information
            articles = data.get("articles", [])
            if not articles:
                print(f"No articles found for NOR {nor}")
            
            for article in articles:
                content = article.get("content", "")
                matches = re.findall(
                    r"(M\.|Mme) ([\w\-]+) ([\w\-]+)(?:,| est nommée?| est nommé) (.*?)(?:,|\.| à compter du) à compter du ([\w\s\-éà]+)(?:\.|;|<br/>)",
                    content,
                    re.IGNORECASE
                )
                if not matches:
                    print(f"No nominees found in article for NOR {nor}")

                for match in matches:
                    genre, prenom, nom, titre, date = match
                    all_nominees.append({
                        "Genre": genre,
                        "Prénom": prenom,
                        "Nom": nom,
                        "Titre": titre.strip(),
                        "Date de début": date.strip(),
                        "Ministère": ministry_name,
                        "Ministre": minister_name,
                        "NOR": nor
                    })
        else:
            print(f"Error with NOR {nor}: {response.status_code} - {response.text}")
            error_nors.append(nor)

    except Exception as e:
        print(f"Exception occurred for NOR {nor}: {e}")
        error_nors.append(nor)

# Convert the collected data into a DataFrame
if all_nominees:
    df = pd.DataFrame(all_nominees)
    print(df)
else:
    print("No valid nominees data found.")

# Log errors for retrying
if error_nors:
    print(f"The following NORs had errors: {error_nors}")

No nominees found in article for NOR PRMX2430415A
No nominees found in article for NOR MOMC2426137A
No nominees found in article for NOR MOMC2426137A
No nominees found in article for NOR MOMC2429204A
No nominees found in article for NOR MOMC2429204A
No nominees found in article for NOR EAEC2428687A
Error with NOR MOMC2429076A: 429 - {"nor": "MOMC2429076A"}
No nominees found in article for NOR PRMX2429339A
No nominees found in article for NOR PRMX2429339A
No nominees found in article for NOR PRMX2429154A
No nominees found in article for NOR PRMX2428951A
No nominees found in article for NOR PRMX2428951A
No nominees found in article for NOR PRMX2428841A
Error with NOR MOMC2427547A: 429 - {"nor": "MOMC2427547A"}
  Genre    Prénom     Nom                                              Titre  \
0   Mme     Sarah   ALLIX        conseillère territoires et services publics   
1   Mme  Juliette   MOLLO  conseillère en charge du marché intérieur et d...   
2   Mme    Esther  GARCIA                c

In [16]:
## Etudions les erreurs en regardant l'output pour les NOR refusés
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Afin de comprendre le format d'un output de nomination, utilisons le NOR d'une nomination donnée
payload = {
    "nor": "PRMX2430415A"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    print(data)  # Affiche le contenu brut de la réponse pour analyse
else:
    print(f"Erreur {response.status_code}: {response.text}")


{'executionTime': 2, 'dereferenced': False, 'id': 'JORFTEXT000050495652_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000050495652', 'title': 'Arrêté du 12 novembre 2024 portant nomination au cabinet de la ministre déléguée auprès du Premier ministre, chargée de la coordination gouvernementale', 'nor': 'PRMX2430415A', 'eli': None, 'alias': None, 'jorfText': 'JORF n°0270 du 15 novembre 2024', 'jurisState': 'Sans état juridique', 'visa': "<p align='left'><br/>La ministre déléguée auprès du Premier ministre, chargée de la coordination gouvernementale,<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000034938597&categorieLien=cid' title='Décret n°2017-1098 du 14 juin 2017'>décret n° 2017-1098 du 14 juin 2017</a> relatif aux collaborateurs du Président de la République et des membres du Gouvernement ;<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000050185112&categorieLien=cid' title='Décret du 5 septembre 2024'>décret du 5 septembre 2024</a> portant nomination du Premier mini

In [ ]:
## CECI marque un nouvel essai, dans lequel on retire des tags html et on a selon Chat une structure
## plus souple : ça n'améliore pour l'isntant pas notre situation

## Plusieurs changements restent à faire : a) régler mon pb, b) avoir plusieurs nominations par NOR

In [23]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# List of NORs to process defined above

# API details
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Initialize lists to track successes and errors
all_nominees = []
error_nors = []

# Loop through each NOR in the list
for nor in nors:
    payload = {"nor": nor}
    try:
        # Rate limit handling
        time.sleep(1)  # Adjust delay to respect the API's rate limits
        
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            data = response.json()

            # Extract ministry name
            ministry_name_match = re.search(r"cabinet du (.*?)(,|$)", data.get("title", ""), re.IGNORECASE)
            ministry_name = ministry_name_match.group(1).strip() if ministry_name_match else "Cabinet"

            # Extract minister's name
            minister_name_match = re.search(r"Fait le .*?\.<br/>(.*?)</p>", data.get("signers", ""), re.DOTALL)
            minister_name = minister_name_match.group(1).strip() if minister_name_match else None

            # Extract nominees information
            articles = data.get("articles", [])
            if not articles:
                print(f"No articles found for NOR {nor}")
                continue
            
            for article in articles:
                content = article.get("content", "")

                # Clean HTML tags from content
                cleaned_content = BeautifulSoup(content, "html.parser").get_text()

                # Skip boilerplate content
                if "sera publié au Journal officiel" in cleaned_content:
                    continue

                # Updated regex for nominees
                matches = re.findall(
                    r"(M\.|Mme)\s+([\w\-]+)\s+([\w\-]+).*?(est nommé|est nommée|:)\s+(.*?)\s+(?:au cabinet.*?|auprès.*?|chargé de.*?|)\s+à compter du\s+([\w\s\-éà]+)",
                    cleaned_content,
                    re.IGNORECASE
                )

                if not matches:
                    print(f"No nominees found in article for NOR {nor}")
                    print("Cleaned content:", cleaned_content)
                    continue

                for match in matches:
                    genre, prenom, nom, _, titre, date = match
                    all_nominees.append({
                        "Genre": genre,
                        "Prénom": prenom,
                        "Nom": nom,
                        "Titre": titre.strip(),
                        "Date de début": date.strip(),
                        "Ministère": ministry_name,
                        "Ministre": minister_name,
                        "NOR": nor
                    })
        else:
            print(f"Error with NOR {nor}: {response.status_code} - {response.text}")
            error_nors.append(nor)

    except Exception as e:
        print(f"Exception occurred for NOR {nor}: {e}")
        error_nors.append(nor)

# Convert the collected data into a DataFrame
if all_nominees:
    df = pd.DataFrame(all_nominees)
    print(df)
else:
    print("No valid nominees data found.")

# Log errors for retrying
if error_nors:
    print(f"The following NORs had errors: {error_nors}")


No nominees found in article for NOR MOMC2426137A
Cleaned content: Est nommé au cabinet du ministre auprès du Premier ministre, chargé des outre-mer :M. Déobal GOBIN, conseiller chargé des relations avec les élus et les acteurs ultramarins, à compter du 18 octobre 2024.
No nominees found in article for NOR MOMC2429204A
Cleaned content: Est nommée au cabinet du ministre auprès du Premier ministre, chargé des outre-mer :Mme Blandine DENBY WILKES, conseillère en charge des relations avec la presse et de la communication, à compter du 4 novembre 2024.
No nominees found in article for NOR MOMC2429076A
Cleaned content: Est nommé au cabinet du ministre auprès du Premier ministre, chargé des outre-mer :M. Jean-Jacques BOREL, conseiller spécial auprès du ministre, à compter du 4 novembre 2024.
No nominees found in article for NOR PRMX2429339A
Cleaned content: Mme Céline Bentz est nommée conseillère procédure et coordination parlementaire au pôle parlementaire du cabinet du Premier ministre, à c